In [ ]:
!pip install wget
from fastai.text import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=e4625bb0bfd9bca36369e4da55609b7466792caf7f0a003cf681aa1f5401c770
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [ ]:
import wget
import os
%tensorflow_version 2.x

import tensorflow as tf
print(tf.__version__)
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

print('Downloading dataset...')

# The URL for the dataset zip file.
url = 'https://nyu-mll.github.io/CoLA/cola_public_1.1.zip'

# Download the file (if we haven't already)
if not os.path.exists('./cola_public_1.1.zip'):
    wget.download(url, './cola_public_1.1.zip')

2.8.0
Found GPU at: /device:GPU:0


In [ ]:
# Unzip the dataset (if we haven't already)
if not os.path.exists('./cola_public/'):
    !unzip cola_public_1.1.zip

Archive:  cola_public_1.1.zip
   creating: cola_public/
  inflating: cola_public/README      
   creating: cola_public/tokenized/
  inflating: cola_public/tokenized/in_domain_dev.tsv  
  inflating: cola_public/tokenized/in_domain_train.tsv  
  inflating: cola_public/tokenized/out_of_domain_dev.tsv  
   creating: cola_public/raw/
  inflating: cola_public/raw/in_domain_dev.tsv  
  inflating: cola_public/raw/in_domain_train.tsv  
  inflating: cola_public/raw/out_of_domain_dev.tsv  


In [ ]:
df = pd.read_csv("./cola_public/raw/in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

Number of training sentences: 8,551



,sentence_source,label,label_notes,sentence
6569,g_81,1,NaN,On which table and under which flower pot did ...
7531,sks13,1,NaN,I noticed John's excessive appreciation of him...
7144,sks13,1,NaN,"This boy must not go to France, but his father..."
3992,ks08,0,*,They want to are kind to her.
5945,c_13,1,NaN,I ate a basket of apples.
5939,c_13,1,NaN,The soldier was given the medal.
5377,b_73,0,*,Sally enough eats caviar.
3954,ks08,1,NaN,Ad agencies call young people Generation X-ers.
6171,c_13,1,NaN,I asked where you found it.
4251,ks08,1,NaN,Harry says that Sally dislikes him.


In [ ]:
#The two properties we actually care about are the the sentence and its label, 
# which is referred to as the "weather it is grammatically correct or not" (0=unacceptable, 1=acceptable).
from sklearn.model_selection import train_test_split

train_sentences = df.sentence
train_labels = df.label
train_set = pd.concat([train_labels,train_sentences], axis=1)
new_train_set, new_val_set= train_test_split(train_set,test_size=0.10,shuffle=False)

In [ ]:
df = pd.read_csv("./cola_public/raw/out_of_domain_dev.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])
test_sentences = df.sentence
test_labels = df.label
new_test_set = pd.concat([test_labels,test_sentences], axis=1)
new_val_set

,label,sentence
7695,1,The moon glows in the darkness.
7696,1,The moon glows.
7697,1,I sang a song with Mary while you did so with ...
7698,1,What Mary did with Bill was sing a song.
7699,1,She tried to leave
...,...,...
8546,0,Poseidon appears to own a dragon
8547,0,Digitize is my happiest memory
8548,1,It is easy to slay the Gorgon.
8549,1,I had the strangest feeling that I knew you.


In [ ]:
#Data preprocessing
import os
print('getcwd:', os.getcwd())
path = os.getcwd()
data_lm = TextLMDataBunch.from_df(path,train_df=new_train_set,valid_df= new_val_set)
data_clas = TextClasDataBunch.from_df(path,train_df=new_train_set, valid_df= new_val_set, vocab=data_lm.train_ds.vocab, bs=32)
data_lm.train_ds.vocab

getcwd: /content


/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(a, dtype=dtype, **kwargs)


/usr/local/lib/python3.7/dist-packages/fastai/text/data.py:124: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sort_idx = np.concatenate(np.random.permutation(ck_idx[1:])) if len(ck_idx) > 1 else np.array([],dtype=np.int)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [ ]:
bs=32
data_lm = load_data(path, 'data_lm_export.pkl', bs=bs)
data_clas = load_data(path, 'data_clas_export.pkl', bs=bs)

/usr/local/lib/python3.7/dist-packages/fastai/text/data.py:124: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sort_idx = np.concatenate(np.random.permutation(ck_idx[1:])) if len(ck_idx) > 1 else np.array([],dtype=np.int)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
torch.cuda.set_device(0)

In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [ ]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.317516,3.629104,0.297098,00:04


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(1e-4, 1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,3.153965,3.436342,0.368973,00:05


In [ ]:
learn.save('cola_language_model')
learn.save_encoder('cola_language_model_encoder')

In [ ]:
#grammar verifier
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('cola_language_model_encoder')
learn.freeze()

In [ ]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.617783,0.594362,0.719626,00:04


/usr/local/lib/python3.7/dist-packages/fastai/text/data.py:124: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sort_idx = np.concatenate(np.random.permutation(ck_idx[1:])) if len(ck_idx) > 1 else np.array([],dtype=np.int)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
data_clas.show_batch()

/usr/local/lib/python3.7/dist-packages/fastai/text/data.py:124: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sort_idx = np.concatenate(np.random.permutation(ck_idx[1:])) if len(ck_idx) > 1 else np.array([],dtype=np.int)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


text,target
"xxbos xxmaj everybody who has ever , worked in any office which contained any xxunk which had ever been used to type any letters which had to be signed by any xxunk who ever worked in any department like mine will know what i xxunk .",1
"xxbos xxmaj hank plays the guitar and finds arrangements for all the old folk songs which are still sung in these hills , and xxmaj ernie writes down all the old folk songs which are still sung in these hills .",1
"xxbos xxmaj in xxmaj january xxunk , a xxunk star in an xxunk xxunk suddenly became xxunk times more xxunk than our xxmaj sun , xxunk making it the xxunk star in our xxunk .",1
"xxbos xxmaj handsome though everyone expects me to try to force xxmaj bill to make xxmaj xxunk agree that xxmaj dick is , i 'm still going to xxmaj marry xxmaj herman .",1
"xxbos xxmaj they said xxmaj tom would pay up , and pay up i 'm sure everybody will tell you that his lawyers expect me to believe he did .",1


In [ ]:
learn.freeze_to(-1)
learn.fit_one_cycle(3, slice(1e-4, 1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.605902,0.607351,0.707944,00:04
1,0.597141,0.622217,0.717290,00:04
2,0.596516,0.601825,0.719626,00:03


/usr/local/lib/python3.7/dist-packages/fastai/text/data.py:124: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sort_idx = np.concatenate(np.random.permutation(ck_idx[1:])) if len(ck_idx) > 1 else np.array([],dtype=np.int)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/fastai/text/data.py:124: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(3, slice(1e-4, 1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.600647,0.590778,0.719626,00:04
1,0.593307,0.587138,0.719626,00:04
2,0.562870,0.604053,0.721963,00:05


/usr/local/lib/python3.7/dist-packages/fastai/text/data.py:124: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sort_idx = np.concatenate(np.random.permutation(ck_idx[1:])) if len(ck_idx) > 1 else np.array([],dtype=np.int)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/fastai/text/data.py:124: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-4, 1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.572190,0.591807,0.719626,00:09
1,0.554499,0.604843,0.721963,00:09
2,0.499379,0.588006,0.724299,00:08
3,0.366990,0.650377,0.728972,00:09
4,0.293698,0.781386,0.738318,00:09


/usr/local/lib/python3.7/dist-packages/fastai/text/data.py:124: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sort_idx = np.concatenate(np.random.permutation(ck_idx[1:])) if len(ck_idx) > 1 else np.array([],dtype=np.int)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/fastai/text/data.py:124: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must

In [ ]:
learn.predict('he said you are')


(Category tensor(1), tensor(1), tensor([0.3475, 0.6525]))

In [ ]:
preds,targs = learn.get_preds(ordered=True)
accuracy(preds,targs)

tensor(0.7383)

In [ ]:
# check if the dataset is inbalanced.
train_labels.value_counts()

1    6023
0    2528
Name: label, dtype: int64

In [ ]:
from sklearn.metrics import matthews_corrcoef
import tensorflow as tf

test_labels_set = []
pred_labels_i_set = []

# Evaluate each test batch using Matthew's correlation coefficient
print('Calculating Matthews Corr. Coef...')

# For each input batch...
for i in range(len(test_labels)):
  logits = learn.predict(test_sentences[i])
  pred_labels_i = 1 if logits[1] == 1 else 0

  test_labels_set.append(test_labels[i])
  pred_labels_i_set.append(pred_labels_i)

# Calculate and store the coef for the test-data. 
matthews = matthews_corrcoef(test_labels_set, pred_labels_i_set) 

print(matthews) 

Calculating Matthews Corr. Coef...
0.2069963591914886


In [ ]:
learn.export()

In [ ]:
from fastai.text import *
learn = load_learner('/content')

In [ ]:
learn.predict('He said')

(Category tensor(1), tensor(1), tensor([0.4113, 0.5887]))

In [ ]:
from google.colab import files
files.download('models') 

In [ ]:
!ls

cola_public	     data_clas_export.pkl  export.pkl  sample_data
cola_public_1.1.zip  data_lm_export.pkl    models


In [ ]:
cd models

/content/models


In [ ]:
!ls

cola_language_model_encoder.pth  cola_language_model.pth


In [ ]:
from google.colab import files
files.download('cola_language_model.pth') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
cd

/root


In [ ]:
!ls

In [ ]:
cd /content

/content


In [ ]:
!ls

cola_public	      data_lm_export.pkl  review_data.csv
cola_public_1.1.zip   export.pkl	  sample_data
data_clas_export.pkl  models		  test_data.csv


In [ ]:
data=pd.read_csv("./review_data.csv")
data.head()

,text,star,app_id,reviewDate
0,Anathi Khanyile,5,com.fingersoft.hillclimb,18/03/21
1,Tony bahut funny hai Hill climbing racing my f...,5,com.fingersoft.hillclimb,18/03/21
2,Teturwu,1,com.fingersoft.hillclimb,18/03/21
3,Hoooooooooooyaaaaaaaaa what a game hoooooooooo...,5,com.fingersoft.hillclimb,18/03/21
4,This game is nice,5,com.fingersoft.hillclimb,18/03/21


In [ ]:
df=data[['text']]
df.head()

,text
0,Anathi Khanyile
1,Tony bahut funny hai Hill climbing racing my f...
2,Teturwu
3,Hoooooooooooyaaaaaaaaa what a game hoooooooooo...
4,This game is nice


In [ ]:
X=df['text'].values
l=[]
for i in X:
  y_pred=learn.predict(i)
  l.append(y_pred)

In [ ]:
df['Label'] = l

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df.tail(50)

,text,Label
29950,bay,"(1, tensor(1), [tensor(0.2267), tensor(0.7733)])"
29951,I love likee so much because i can help and su...,"(1, tensor(1), [tensor(0.0400), tensor(0.9600)])"
29952,Cool,"(1, tensor(1), [tensor(0.2128), tensor(0.7872)])"
29953,Likee bhout acha hai,"(1, tensor(1), [tensor(0.0007), tensor(0.9993)])"
29954,Super app,"(1, tensor(1), [tensor(0.1256), tensor(0.8744)])"
29955,Good,"(1, tensor(1), [tensor(0.1048), tensor(0.8952)])"
29956,Mujeebriyaz,"(1, tensor(1), [tensor(0.2128), tensor(0.7872)])"
29957,tnQ app,"(1, tensor(1), [tensor(0.1698), tensor(0.8302)])"
29958,I have likee and I love it. I got it because m...,"(1, tensor(1), [tensor(0.0813), tensor(0.9187)])"
29959,It is good app I rate it 5 stars,"(1, tensor(1), [tensor(0.0213), tensor(0.9787)])"


In [ ]:
df.to_csv('review_output.csv') 